In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth

import time
import random
import string
import json
import pafy

import pandas as pd
import numpy as np

# Hide video unavailable warnings
import warnings
warnings.filterwarnings("ignore")

# Data Inladen

In [2]:
mails = pd.read_csv("../data/mails.csv")

In [3]:
videos = pd.read_csv("../data/uploads.csv")

# Helpers

In [4]:
def vid_to_watch(videos):
    """Finds a conspiracy video to watch and 
    makes sure it's still accessible and under 2 hours long"""
    
    # Choose a conspiracy video to watch
    to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
        
    # Check its length
    url = f"http://www.youtube.com/watch?v={to_watch}"
    try:
        video = pafy.new(url);
        vid_len = video.length
    except: # Video no longer accessible
        vid_len = 9999
    
    # Videos over 2 hours will be skipped
    while vid_len > 7200:
        to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
        
        url = f"http://www.youtube.com/watch?v={to_watch}"
        try:
            video = pafy.new(url);
            vid_len = video.length
        except:
            vid_len = 9999
    
    return to_watch

In [5]:
def init_driver():
    """Initializes a selenium driver in stealth mode"""
        
    # Add options
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")

    options.add_argument("start-maximized")

    # options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # Start driver
    driver = webdriver.Chrome(options=options)

    # Hide the fact we're using a bot
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    
    return driver

In [6]:
def login_google(driver, mail, password):
    """Logs in on Google given a driver, email-address and password.
    This code only works if the email-address was made within chromedriver itself"""
    
    # login on google
    driver.get("https://accounts.google.com/ServiceLogin")
    time.sleep(np.random.uniform(1, 2.5))

    for letter in mail:
        driver.find_element_by_id("identifierId").send_keys(letter)
        time.sleep(np.random.uniform(0.05, 0.2))

    driver.find_element_by_id("identifierNext").click()

    time.sleep(np.random.uniform(1, 2))

    driver.find_element_by_xpath("//input[@name = 'password']").send_keys(password)
    driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

    time.sleep(np.random.uniform(1, 2))

    # If security-check is asked, skip it
    try:
        driver.find_element_by_xpath("//div[contains(@class, 'U26fgb O0WRkf')]").click()
    except:
        pass

# Het experiment

In [19]:
experiment_results = {"user":[], "vids_watched":[], "video":[], "channel":[]}

for user in mails.itertuples():
    # Find current user's info
    mail, password = user[1], user[2]

    # Initialize driver
    driver = init_driver()    

    # Login to google
    login_google(driver, mail, password)

    # Start watching videos
    for i in range(1, 3):
        # Get a random video from the dataset and watch it
        to_watch = vid_to_watch(videos)
        driver.get(f"http://youtube.com/watch?v={to_watch}")
        vid_running = True

        # Skip ad(s)
        for ad in range(2):
            time.sleep(7)
            try:
                driver.find_element_by_xpath("//div[@class = 'ytp-ad-text ytp-ad-skip-button-text']").click()
            except:
                pass

        # Toggle autoplayer
        time.sleep(2)
        autoplay = driver.find_element_by_xpath("//div[@class = 'ytp-autonav-toggle-button']")

        if autoplay.get_attribute("aria-checked") == "true":
            autoplay.click()

        # Watch video and check if it is done every second
        while vid_running:
            try:
                driver.find_element_by_xpath("//div[contains(@class, 'ended-mode')]")
                vid_running = False
            except:
                pass

            time.sleep(1)

        # Go to the youtube homepage
        driver.get("http://youtube.com")
        time.sleep(1)

        # Get videos on youtube home
        channels = driver.find_elements_by_xpath("//a[@id = 'avatar-link']")
        vids = driver.find_elements_by_xpath("//a[@id = 'video-title-link']")

        for rec in range(len(channels)):
            experiment_results["user"].append(mail)
            experiment_results["vids_watched"].append(i)
            
            experiment_results["video"].append(vids[rec].get_attribute("href"))
            experiment_results["channel"].append(channels[rec].get_attribute("href"))
    break
            
pd.DataFrame(experiment_results)

ERROR: Private video
Sign in if you've been granted access to this video
ERROR: Video unavailable
This video is no longer available because the YouTube account associated with this video has been terminated.


,user,vids_watched,video,channel
0,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=U3otWaBrvEc,https://www.youtube.com/c/Campbellteaching
1,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=uZdv-TtiMkg,https://www.youtube.com/thewayofthemaster
2,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=0S41RYQbuKQ,https://www.youtube.com/user/youversion
3,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=IzbrztZFCFA,https://www.youtube.com/user/schattenwolff
4,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=e1eFMzO3iPc,https://www.youtube.com/user/PBSNewsHour
5,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=xfN_S5W0MJU,https://www.youtube.com/c/bugoutbrothers
6,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=cmXA4VT2mUE,https://www.youtube.com/c/myJLI
7,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=YSidv4ZiKkk,https://www.youtube.com/channel/UCGnsRwFS8Xm13...
8,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=_jX1TYHiAmg,https://www.youtube.com/channel/UChXRU1noMzd2w...
9,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=VajG7HhmKaw,https://www.youtube.com/channel/UCeDzaWX98ne38...
